<a href="https://colab.research.google.com/github/MNabildawami/deteksiface/blob/main/deteksiface.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import cv2
import dlib
import os
import pandas as pd

!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bzip2 -d shape_predictor_68_face_landmarks.dat.bz2

# Muat model detektor wajah dlib
detector = dlib.get_frontal_face_detector()

# Muat model prediktor landmark wajah (pre-trained model)
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# Fungsi untuk mengekstrak landmark dari wajah dan mengembalikan sebagai list data
def extract_face_landmarks_data(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    # Periksa apakah wajah terdeteksi
    if len(faces) == 0:
        print("Tidak ada wajah yang terdeteksi.")
        return None

    landmarks_data = []  # List untuk menyimpan data landmark dari satu gambar

    for face in faces:
        landmarks = predictor(gray, face)
        # Loop melalui 68 titik landmark
        for n in range(0, 68):
            x = landmarks.part(n).x
            y = landmarks.part(n).y
            landmarks_data.append((x, y))  # Tambahkan koordinat (x, y) ke list
    return landmarks_data

# Fungsi untuk memproses semua gambar dalam folder dan mengembalikan dataframe dengan landmark
def process_folder_images(folder_path):
    data = []

    # Loop untuk setiap file gambar di folder
    for file_name in os.listdir(folder_path):
        if file_name.endswith(('.jpg', '.jpeg', '.png')):  # Filter hanya file gambar
            image_path = os.path.join(folder_path, file_name)
            image = cv2.imread(image_path)

            # Cek apakah gambar berhasil dimuat
            if image is None:
                print(f"Gambar {file_name} tidak ditemukan atau tidak bisa dimuat!")
                continue

            # Ekstraksi fitur geometri sebagai data
            landmarks_data = extract_face_landmarks_data(image)

            if landmarks_data is not None:
                # Tambahkan hasil ekstraksi ke list data, termasuk nama file
                flattened_landmarks = [coord for point in landmarks_data for coord in point]  # Flatten koordinat
                data.append([file_name] + flattened_landmarks)

    # Buat kolom untuk dataframe
    column_names = ['image_name'] + [f'x{i+1}' for i in range(68)] + [f'y{i+1}' for i in range(68)]

    # Buat dataframe dari data
    df = pd.DataFrame(data, columns=column_names)

    return df

# Tentukan folder yang berisi gambar
folder_path = '/content/drive/MyDrive/footage item pengpol /Fajri'  # Ganti dengan path folder Anda

# Proses semua gambar di folder dan simpan hasil ekstraksi ke dataframe
landmarks_df = process_folder_images(folder_path)

# Simpan hasil ekstraksi ke file CSV
output_csv = 'landmarks_output.csv'
landmarks_df.to_csv(output_csv, index=True)

# Tampilkan dataframe sebagai hasil ekstraksi
print(landmarks_df)

# Simpan hasil ekstraksi ke file Excel
output_excel = 'landmarks_output.xlsx'
landmarks_df.to_excel(output_excel, index=True)  # Simpan ke Excel

# Tampilkan pesan sukses

print(f"Data landmark berhasil disimpan dalam file Excel: {output_excel}")

--2024-12-12 19:09:30--  http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 64040097 (61M)
Saving to: ‘shape_predictor_68_face_landmarks.dat.bz2.6’

shape_predictor_68_ 100%[===================>]  61.07M  34.7MB/s    in 1.8s    

2024-12-12 19:09:32 (34.7 MB/s) - ‘shape_predictor_68_face_landmarks.dat.bz2.6’ saved [64040097/64040097]

bzip2: Output file shape_predictor_68_face_landmarks.dat already exists.
           image_name   x1   x2   x3   x4   x5   x6   x7    x8   x9  ...  y59  \
0  Image_99222729.jpg  205  735  203  786  205  839  210   891  222  ...  432   
1  Image_9d9759cb.jpg   69  656   75  743   88  830  109   913  145  ...  418   
2  Image_8245f3b2.jpg   70  607   76  705   97  801  126   893  167  ...  427   
3  Image_2fbe07b9.jpg  106  630  110  713  118  797  140   878  180  ...  464   
4  I